# Integer Neural Network Implementation
In the following Jupyter Notebook an Integer Neural Network Implementation is presented. Neural Networks are commonly implemented using floating point arithmetic, but if we want to implement it into hardware, the use of integer arithmetic will be more convenient. When making an integer implementation many issues can arise, such as overflow, signed arithmetic, function implementations (i.e. sigmoid), etc. 

In this Jupyter Notebook the following hardware issues are covered:
* Neural Network Architecture
* Neurons on hardware
* Forward pass
* Cost function
* Backward pass
* Parameter update
* How to test your design

## Neural Network Architecture
The Neural Network architecture developed here is a three layer network, with two hidden layers and one input and one output layer. This simple architecture is shown in the following image.

![Neural Network](Images/nn_architecture.png)

Usually when making Neural Networks on hardware training is performed on software, and inference is performed on hardware. There are many reasons why training is preferred to be done on software such as:
* Backpropagation algorithm is easier to implement
* Activation functions are easily performed on software
* Regularization, such as L2, is simple to implement
* Optimization algorithms, like Adam an RMSprop, are hard to develop on hardware

In this work we propose to make both **training and inference on hardware**. In the training case, we propose to perform backpropagation and activation functions on hardware. Regularization and optimization algorithms are not performed, but the used of bit shifting can have similar effects as a regularization technique.

## Neurons on Hardware

If you look at the image, you can see that the number of connections a neuron has depends on the previous number of neurons on the previous layer. This situation is not that important when you deal with floating point data, but if you work with integer numbers, the number of connections is limited by the integer precision. 

To understand this, consider how we multiply two integer numbers, $x$ and $w$ of size $N$, then $wx$ would be of size $2N$. Now if you make a calculation like this $w_{1}x_{1}+w_{2}x_{2}+ \cdots w_{k}x_{k}$ the size would be $\log_{2}(k)+2N$.

![Multiplication bit size](Images/bit_size_mult.png)

What this means is that if you have a single neuron, you can connect up to $k$ inputs to a neuron, and as trade-off you will add $\log_{2}(k)$ bits. You have to be careful on how many inputs can be stacked together before having an overflow. 
Now if you consider a single neuron as a two step calculation such as:

1. Calculation of output $z$
```python
z = w1*x1+w2*x2+...+wk*xk
```
2. Calculation of activation $a$
```python
a = sigmoid(z)
```
![Single neuron multiple connection](Images/single_neuron_multiple_connection.png)

The activation function must be limited to a correct bit size. We can perform this task by shifting our $z$ value to the right $N$ times, which in turn makes our input activation values lower to handle. With this our activation function maps from $\log_{2}(k)+N$ to $N$ bits.

![Activation bit size](Images/bit_size_activ.png)

### How can we implement a single neuron on hardware?
We don't implement a single neuron, we implement a layer of neurons and make use of the ideas previously presented. To do so, we will use *System Verilog*. SystemVerilog is a superset of *Verilog* language and is getting more widely adopted in industry. It is an IEEE standard and supports lot of enhancements from Verilog for design constructs. In addition SystemVerilog supports all features for Verification - including OOPs , assertions, coverage etc , thus making a single language suitable for both design and verification. If you know Verilog you will easily make the move to System Verilog.

First we want to achieve the following calculation:

```python
# W weight matrix
# X input vector
# b bias vector
# sigmoid and relu are activation functions
Z = W*X+b
if('sigmoid_activation'):
    A=sigmoid(Z)
elif('relu_activation'):
    A=relu(Z)
```
As we already mention, we divided this problem into two parts:

#### 1. Calculation of output $z$
We further divide this problem into two parts:
##### 1.1 Calculation of product $W\cdot X$
 
Remember when you multiply a matrix $W$ and a vector $X$ you can perform this calculation as a recursive MAC operation as follows:

```python
# M is row size
# N is column size
for i in M:
    for j in N:
        y[i][0] = y[i][0] + W[i][j]*X[j][0]

```

This can be achieve in System Verilog as:

```c
module mult #(parameter M, parameter N)
    ...
// B is a temporal matrix that stores vector-matrix products
always_ff @(posedge clk) begin
        for(i=0;i<=M-1;i=i+1)
            for(j=0; j<=N-1; j=j+1)
                B[i][j] = A[i][j]*x[j][0];  
    end 

...
        
// y is a vector that sums all B values in the vertical axis
genvar i, j;
        generate
            for(i=0;i<M;i=i+1) begin
            assign y[i][0] = B[i][0] + B[i][1];
                for(j=0; j<=N-3; j=j+1)
                    always_ff @(posedge clk) begin
                            y[i][j+1] <= y[i][j] + B[i][j+2];
                        end
           end     
        endgenerate 
endmodule

```

One of the language most powerful statements is the `generate` statement because it allow us the create hardware designs in a recursive way as this example.

![Multiplier](Images/multiplier.png)

##### 1.2 Addition of $b$ to product $W \cdot X$

```python
# M is row size
for i in M:
    Z[i][0] = Z[i][0] + b[i][0]

```

This can also be achieved using System Verilog as follows:

```C
module bias #(parameter M)
    ...
genvar i;
     generate
         for (i=0; i<M; i=i+1) begin
             always_ff @(posedge clk) begin
                if(!reset) a[i][0] <= 0;
                else begin
                    a[i][0]<=b[i][0] + z[i][0]; 
                end
             end
         end
     endgenerate
endmodule
```


![Bias](Images/bias.png)

#### 2. Calculation of activation $a$
For the activation function, two types were considered:
##### 2.1 RELU Unit
For this type of function we need to implement the following calculation:
```python
A = maximum(0, Z)
```
Since this is just a clipping of values lower than zero, this can be done by implementing this:
```c
module relu #(parameter M = 5)
...
    
 genvar i;
     generate
         for (i=0; i<M; i=i+1) begin
             always_ff @(posedge clk) begin
                if(!reset) a[i][0] <= 0;
                else begin
                    if(z[i][0]<0) a[i][0]<= 0;
                    else a[i][0]<=z[i][0]; 
                end
             end
         end
     endgenerate
    
endmodule
```

![Relu](Images/relu.png)
##### 2.2 Sigmoid Unit
For this type of unit, a table implementation was performed. On real numbers Relu performs this calculation:
$$ \sigma(z)=\frac{1}{1-e^{-z}}$$ 
As discussed before, our input values are of size $\log_{2}(k)+N$, which means that when mapping we have to be careful to cover this range of inputs. The actual table implementation maps $\sigma(z)$ to $N$ bits by calculating a table for all possible values $z$ like this:

$$ \sigma(z)=\frac{1}{1-e^{-z/(\log_{2}(k)+N)}}$$ 

Table size can be reduced if redundancy was removed. For this case a 96% reduction in size was achieved if using *if-else* clauses to create this table.

## Forward pass
For a Neural Network, the implementation of a forward pass is just the calculation of this equation:

$$ A = \sigma(WX+b)$$

and pass vector $A$ to the next layer of neurons.

So an implementation might look like this:

```python
def forward(W,X,b):
    return activation(W*X + b)

A1 = forward(W1,X1,b1)
A2 = forward(W2,A1,b2)
AL = forward(W3,A2,b3)

```

A hardware implementation will make use of the previous hardware blocks as follows:

```c
// M number of neurons
// N input size or previous number of neurons

module forward_neurons #(parameter M, parameter N) 
    ...    
mult #(M, N) multiplier(.clk(clk), .A(A), .x(x), .reset(reset), .b(Ax));
bias #(M) add_bias(.clk(clk), .z(Ax), .b(b), .reset(reset), .a(z)); 
relu #(M) activation(.clk(clk), .z(z), .reset(reset), .a(a));  
    
endmodule 
```

And then make an instance of those modules
```c
module forward_net(
    ...
    // Forward Neuron Network
    forward_neurons #(L2, L1) layer_1(clk, W1, a1, b1, reset, a2);
    forward_neurons #(L3, L2) layer_2(clk, W2, a2, b2, reset, a3);
    forward_neurons #(L4, L3) layer_3(clk, W3, a3, b3, reset, a4);
    
    ...
 endmodule
 ```
 
 ![Forward](Images/forward.png)

## Loss and Cost Function
To calculate our cost, we will use the cross entropy loss function defined as:
$$L = -y\log{\hat{y}}-(1-y)\log{(1-\hat{y})}$$

As implemented for the sigmoid function, we also applied a logarithm table, but in this case there is a one on one mapping, therefore no table reduction was achieved.

Cost, can be calculated on two different ways:
1. Using stochastic gradient descent, SGD
2. Gradient descent, GD

If using SGD cost is calculated on every input for every epoch, on GD cost is calculated on average for each epoch. In terms of hardware design SGD is easier to implement, but raises convergence issues.

For the gradient of the loss, we also create a table that performs this calculation:
$$\frac{dL}{dA} = -\frac{y}{\hat{y}}+\frac{(1-y)}{(1-\hat{y})}$$

## Backward pass
Backward pass works by using gradient descent to solve the problem of reducing the cost of our network. When implementing backward propagation, two main blocks can be defined:
1. Activation backward
2. Linear backward

as shown in the image.

![Backward](Images/backward.png)

### 1. Activation backward
Activation backward depends on the type of activation function and calculates the derivative of the cost with respect of its output. 

#### 1.1 Relu activation backward
The implementation of relu backward is very simple, and can be implemented as:
```python
    # dZ is the derivative of loss with respect to Z
    # dA is the derivative of loss with respect to A

    if(Z>0):
        dZ = 1*dA
    else:
        dZ = 0*dA
```

#### 1.2 Sigmoid activation backward
The implementation of sigmoid backward is more complex, and it introduces the calculation of the following:
$$\frac{dL}{dZ}=\frac{dL}{dA}s(1-s)$$

```python
# dZ is the derivative of loss with respect to Z
# dA is the derivative of loss with respect to A

    s = 1/(1+np.exp(-Z))
    dZ = dA * s * (1-s)
```

Hardware implementations can be done reusing the sigmoid table and taking care of the bit size as it increases due to the three multiplications involved.

### 2. Linear backward
Linear backward calculation can be performed in software as follows:

```python
# dW is the derivative of loss with respect to W
# db is the derivative of loss with respect to b
# dA_prev is the derivative of loss with respect to A_(L-1)

dW = dZ*transpose(A)
db = dZ
dA_prev = transpose(W)*dZ
```

 ![Backward](Images/backdrop_hw.png)

## Parameter update
Parameter update is implemented by doing the following calculations:

$$W = W - \alpha \frac{dL}{dW}$$

$$b = b - \alpha \frac{dL}{db}$$

Where $\alpha$ is the learning rate of our network. If our learning rate is to big the network might not converge, and if the learning rate is to small it might learn very slowly.

To make this implementation in hardware, our learning rate is just a number that shifts $\frac{dL}{dW}$ or $\frac{dL}{db}$ to the right, this in turn reduces the possible learning rates that can be performed but gives us a very simple hardware.

```c
// y is the parameter to update
// dx is the gradient
// learning_rate is a number
assign y[i][j] = x[i][j] + (dx[i][j]>>learning_rate);   
```

## Final Thoughts
When you develop complex hardware designs there are many issues that can go wrong, therefore a good methodology is always good. Here are some guidelines than can help you:

1. Write your ideas in paper and come with an architecture.
2. Simulate your project in software, and solve the simple case.
3. Try to make your software simulation into hardware.
4. Make good software practices when writing software and hardware code.
5. If you can make scripts, do it. They will save you time.